In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import seaborn as sns
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [2]:
df = pd.read_csv(r"C:\Users\Ben\Desktop\moviedata\proj2train.csv", header=0) 
df.head()
test = pd.read_csv(r"C:\Users\Ben\Desktop\moviedata\proj2test.csv")

In [3]:
test.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew
0,3001,"[{'id': 34055, 'name': 'Pokémon Collection', '...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",http://www.pokemon.com/us/movies/movie-pokemon...,tt1226251,ja,ディアルガVSパルキアVSダークライ,Ash and friends (this time accompanied by newc...,3.851534,...,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",7/14/07,90.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Somewhere Between Time & Space... A Legend Is ...,Pokémon: The Rise of Darkrai,"[{'id': 11451, 'name': 'pok√©mon'}, {'id': 115...","[{'cast_id': 3, 'character': 'Tonio', 'credit_...","[{'credit_id': '52fe44e7c3a368484e03d683', 'de..."
1,3002,NaN,88000,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,tt0051380,en,Attack of the 50 Foot Woman,When an abused wife grows to giant size becaus...,3.559789,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",5/19/58,65.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A titanic beauty spreads a macabre wave of hor...,Attack of the 50 Foot Woman,"[{'id': 9748, 'name': 'revenge'}, {'id': 9951,...","[{'cast_id': 2, 'character': 'Nancy Fowler Arc...","[{'credit_id': '55807805c3a3685b1300060b', 'de..."
2,3003,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,tt0118556,en,Addicted to Love,Good-natured astronomer Sam is devastated when...,8.085194,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",5/23/97,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Comedy About Lost Loves And Last Laughs,Addicted to Love,"[{'id': 931, 'name': 'jealousy'}, {'id': 9673,...","[{'cast_id': 11, 'character': 'Maggie', 'credi...","[{'credit_id': '52fe4330c3a36847f8041367', 'de..."
3,3004,NaN,6800000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",http://www.sonyclassics.com/incendies/,tt1255953,fr,Incendies,A mother's last wishes send twins Jeanne and S...,8.596012,...,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",9/4/10,130.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,The search began at the opening of their mothe...,Incendies,"[{'id': 378, 'name': 'prison'}, {'id': 539, 'n...","[{'cast_id': 6, 'character': 'Nawal', 'credit_...","[{'credit_id': '56478092c3a36826140043af', 'de..."
4,3005,NaN,2000000,"[{'id': 36, 'name': 'History'}, {'id': 99, 'na...",NaN,tt0418753,en,Inside Deep Throat,"In 1972, a seemingly typical shoestring budget...",3.217680,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2/11/05,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It was filmed in 6 days for 25 thousand dollar...,Inside Deep Throat,"[{'id': 279, 'name': 'usa'}, {'id': 1228, 'nam...","[{'cast_id': 1, 'character': 'Narrator (voice)...","[{'credit_id': '52fe44ce9251416c75041967', 'de..."


In [4]:
df['year'] = pd.DatetimeIndex(df['release_date']).year
test['year'] = pd.DatetimeIndex(test['release_date']).year

In [5]:
df['revenue'] = np.log1p(df['revenue'])

In [6]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [7]:
missing_values_table(df)
# we have a good amount of missing variables that have to be either dropped or filled as it is a kaggle competition and we cannot 
# drop rows on the test set

Your selected dataframe has 24 columns.
There are 13 columns that have missing values.


,Missing Values,% of Total Values
belongs_to_collection,2396,79.9
homepage,2054,68.5
tagline,597,19.9
Keywords,276,9.2
production_companies,156,5.2
production_countries,55,1.8
spoken_languages,20,0.7
crew,16,0.5
cast,13,0.4
overview,8,0.3


In [8]:
#df["has_colection"] = df['belongs_to_collection'].apply(lambda x: lambda: len(x) if x != {} else 0)

In [9]:
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue,year
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",16.326300,2015
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",18.370959,2004
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",16.387512,2014
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16.588099,2012
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",15.182615,2009


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     4398 non-null   int64  
 1   belongs_to_collection  877 non-null    object 
 2   budget                 4398 non-null   int64  
 3   genres                 4382 non-null   object 
 4   homepage               1420 non-null   object 
 5   imdb_id                4398 non-null   object 
 6   original_language      4398 non-null   object 
 7   original_title         4398 non-null   object 
 8   overview               4384 non-null   object 
 9   popularity             4398 non-null   float64
 10  poster_path            4397 non-null   object 
 11  production_companies   4140 non-null   object 
 12  production_countries   4296 non-null   object 
 13  release_date           4397 non-null   object 
 14  runtime                4394 non-null   float64
 15  spok

In [12]:
#not used
df = df.drop(["homepage","belongs_to_collection", "poster_path", 'status', 'tagline','Keywords'], axis = 1)
test = test.drop(["homepage","belongs_to_collection", "poster_path", 'status','tagline','Keywords'], axis = 1)


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    4398 non-null   int64  
 1   budget                4398 non-null   int64  
 2   genres                4382 non-null   object 
 3   imdb_id               4398 non-null   object 
 4   original_language     4398 non-null   object 
 5   original_title        4398 non-null   object 
 6   overview              4384 non-null   object 
 7   popularity            4398 non-null   float64
 8   production_companies  4140 non-null   object 
 9   production_countries  4296 non-null   object 
 10  release_date          4397 non-null   object 
 11  runtime               4394 non-null   float64
 12  spoken_languages      4356 non-null   object 
 13  title                 4395 non-null   object 
 14  cast                  4385 non-null   object 
 15  crew                 

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                2993 non-null   object 
 3   imdb_id               3000 non-null   object 
 4   original_language     3000 non-null   object 
 5   original_title        3000 non-null   object 
 6   overview              2992 non-null   object 
 7   popularity            3000 non-null   float64
 8   production_companies  2844 non-null   object 
 9   production_countries  2945 non-null   object 
 10  release_date          3000 non-null   object 
 11  runtime               2998 non-null   float64
 12  spoken_languages      2980 non-null   object 
 13  title                 3000 non-null   object 
 14  cast                  2987 non-null   object 
 15  crew                 

In [15]:
df['cast'].fillna('0',inplace=True)
df['crew'].fillna('0',inplace=True)

test['cast'].fillna('0',inplace=True)
test['crew'].fillna('0',inplace=True)
# since we count crew and cast alter it is fine to just fill with as we it wont affect the feature

In [16]:
print (df['genres'].mode()[0])
print (df['spoken_languages'].mode()[0])
print (df['production_companies'].mode()[0])
print (df['production_countries'].mode()[0])

[{'id': 18, 'name': 'Drama'}]
[{'iso_639_1': 'en', 'name': 'English'}]
[{'name': 'Paramount Pictures', 'id': 4}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]


In [17]:
df['genres'].fillna(df['genres'].mode()[0], inplace=True)
df['spoken_languages'].fillna(df['spoken_languages'].mode()[0], inplace=True)
df['production_companies'].fillna(df['production_companies'].mode()[0], inplace=True)
df['production_countries'].fillna(df['production_countries'].mode()[0], inplace=True)
#test
test['genres'].fillna(test['genres'].mode()[0], inplace=True)
test['spoken_languages'].fillna(test['spoken_languages'].mode()[0], inplace=True)
test['production_companies'].fillna(test['production_companies'].mode()[0], inplace=True)
test['production_countries'].fillna(test['production_countries'].mode()[0], inplace=True)
#print (df['spoken_languages'].mode()[0])
#print (df['production_companies'].mode()[0])
#print (df['production_countries'].mode()[0])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                3000 non-null   object 
 3   imdb_id               3000 non-null   object 
 4   original_language     3000 non-null   object 
 5   original_title        3000 non-null   object 
 6   overview              2992 non-null   object 
 7   popularity            3000 non-null   float64
 8   production_companies  3000 non-null   object 
 9   production_countries  3000 non-null   object 
 10  release_date          3000 non-null   object 
 11  runtime               2998 non-null   float64
 12  spoken_languages      3000 non-null   object 
 13  title                 3000 non-null   object 
 14  cast                  3000 non-null   object 
 15  crew                 

In [19]:
missing_values_table(df)

Your selected dataframe has 18 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
overview,8,0.3
runtime,2,0.1


In [20]:
df["overview"].fillna(0)
test["overview"].fillna(0)

0       Ash and friends (this time accompanied by newc...
1       When an abused wife grows to giant size becaus...
2       Good-natured astronomer Sam is devastated when...
3       A mother's last wishes send twins Jeanne and S...
4       In 1972, a seemingly typical shoestring budget...
                              ...                        
4393    A young man is plunged into a life of subterfu...
4394    Three friends, whose lives have been drifting ...
4395    Frank Galvin is a down-on-his luck lawyer, red...
4396    For 19-year-old Jay, fall should be about scho...
4397    Twelve episodic tales in the life of a Parisia...
Name: overview, Length: 4398, dtype: object

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                3000 non-null   object 
 3   imdb_id               3000 non-null   object 
 4   original_language     3000 non-null   object 
 5   original_title        3000 non-null   object 
 6   overview              2992 non-null   object 
 7   popularity            3000 non-null   float64
 8   production_companies  3000 non-null   object 
 9   production_countries  3000 non-null   object 
 10  release_date          3000 non-null   object 
 11  runtime               2998 non-null   float64
 12  spoken_languages      3000 non-null   object 
 13  title                 3000 non-null   object 
 14  cast                  3000 non-null   object 
 15  crew                 

In [22]:
test['runtime'].fillna(test['runtime'].mean(),inplace=True)
df['runtime'].fillna(test['runtime'].mean(),inplace=True)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                3000 non-null   object 
 3   imdb_id               3000 non-null   object 
 4   original_language     3000 non-null   object 
 5   original_title        3000 non-null   object 
 6   overview              2992 non-null   object 
 7   popularity            3000 non-null   float64
 8   production_companies  3000 non-null   object 
 9   production_countries  3000 non-null   object 
 10  release_date          3000 non-null   object 
 11  runtime               3000 non-null   float64
 12  spoken_languages      3000 non-null   object 
 13  title                 3000 non-null   object 
 14  cast                  3000 non-null   object 
 15  crew                 

In [24]:
missing_values_table(test)

Your selected dataframe has 17 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
overview,14,0.3
title,3,0.1
release_date,1,0.0
year,1,0.0


In [25]:
print(test[test['release_date'].isnull()])
print(test[test['year'].isnull()])


       id  budget                         genres    imdb_id original_language  \
828  3829       0  [{'id': 18, 'name': 'Drama'}]  tt0210130                en   

                 original_title  \
828  Jails, Hospitals & Hip-Hop   

                                              overview  popularity  \
828  Jails, Hospitals &amp; Hip-Hop is a cinematic ...    0.009057   

                          production_companies  \
828  [{'name': 'Paramount Pictures', 'id': 4}]   

                                  production_countries release_date  runtime  \
828  [{'iso_3166_1': 'US', 'name': 'United States o...          NaN     90.0   

                             spoken_languages                       title  \
828  [{'iso_639_1': 'en', 'name': 'English'}]  Jails, Hospitals & Hip-Hop   

    cast crew  year  
828   []    0   NaN  
       id  budget                         genres    imdb_id original_language  \
828  3829       0  [{'id': 18, 'name': 'Drama'}]  tt0210130                en   

 

In [26]:
test['release_date'].fillna('3/20/01',inplace=True)

In [27]:
test['year'].fillna("2000", inplace = True)

In [28]:
test['title'].fillna(test['original_title'],inplace=True)

In [29]:
missing_values_table(test)

Your selected dataframe has 17 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
overview,14,0.3


In [30]:
df.info()
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                3000 non-null   object 
 3   imdb_id               3000 non-null   object 
 4   original_language     3000 non-null   object 
 5   original_title        3000 non-null   object 
 6   overview              2992 non-null   object 
 7   popularity            3000 non-null   float64
 8   production_companies  3000 non-null   object 
 9   production_countries  3000 non-null   object 
 10  release_date          3000 non-null   object 
 11  runtime               3000 non-null   float64
 12  spoken_languages      3000 non-null   object 
 13  title                 3000 non-null   object 
 14  cast                  3000 non-null   object 
 15  crew                 

In [31]:
#df = df.dropna()

In [32]:
#missing_values_table(df)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                3000 non-null   object 
 3   imdb_id               3000 non-null   object 
 4   original_language     3000 non-null   object 
 5   original_title        3000 non-null   object 
 6   overview              2992 non-null   object 
 7   popularity            3000 non-null   float64
 8   production_companies  3000 non-null   object 
 9   production_countries  3000 non-null   object 
 10  release_date          3000 non-null   object 
 11  runtime               3000 non-null   float64
 12  spoken_languages      3000 non-null   object 
 13  title                 3000 non-null   object 
 14  cast                  3000 non-null   object 
 15  crew                 

In [34]:
df['year'] = pd.DatetimeIndex(df['release_date']).year
test['year'] = pd.DatetimeIndex(test['release_date']).year

In [35]:
Y = df["revenue"]

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                3000 non-null   object 
 3   imdb_id               3000 non-null   object 
 4   original_language     3000 non-null   object 
 5   original_title        3000 non-null   object 
 6   overview              2992 non-null   object 
 7   popularity            3000 non-null   float64
 8   production_companies  3000 non-null   object 
 9   production_countries  3000 non-null   object 
 10  release_date          3000 non-null   object 
 11  runtime               3000 non-null   float64
 12  spoken_languages      3000 non-null   object 
 13  title                 3000 non-null   object 
 14  cast                  3000 non-null   object 
 15  crew                 

In [37]:
Y.head()

0    16.326300
1    18.370959
2    16.387512
3    16.588099
4    15.182615
Name: revenue, dtype: float64

In [38]:
#df["IsCollection"].value_counts()

In [39]:
df.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,title,cast,crew,revenue,year
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Hot Tub Time Machine 2,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",16.326300,2015
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",The Princess Diaries 2: Royal Engagement,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",18.370959,2004
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Whiplash,"[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",16.387512,2014
3,4,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Kahaani,"[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16.588099,2012
4,5,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Marine Boy,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",15.182615,2009


In [40]:
df["budget_year_ratio"] = df['budget'] / (df['year'] * df['year'])
test["budget_year_ratio"] = test['budget'] / (test['year'] * test['year'])

In [41]:
df.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,title,cast,crew,revenue,year,budget_year_ratio
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Hot Tub Time Machine 2,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",16.326300,2015,3.448085
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",The Princess Diaries 2: Royal Engagement,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",18.370959,2004,9.960120
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Whiplash,"[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",16.387512,2014,0.813570
3,4,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Kahaani,"[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16.588099,2012,0.296432
4,5,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Marine Boy,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",15.182615,2009,0.000000


In [42]:
#little function for for finding an overview
def extractOverviewInfo(df):
    df['has_overview'] = df['overview'].apply(lambda x: isinstance(x, str))
    df['overview_len'] = df['overview'].apply(lambda x: len(x) if isinstance(x, str) else 0)
    return df

In [43]:
df = extractOverviewInfo(df)


In [44]:
test = extractOverviewInfo(test)

In [45]:
df['has_overview'] = df['has_overview'].astype(int)
test['has_overview'] = test['has_overview'].astype(int)
df = df.drop(['overview'], axis = 1)
test = test.drop(['overview'], axis = 1)


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                3000 non-null   object 
 3   imdb_id               3000 non-null   object 
 4   original_language     3000 non-null   object 
 5   original_title        3000 non-null   object 
 6   popularity            3000 non-null   float64
 7   production_companies  3000 non-null   object 
 8   production_countries  3000 non-null   object 
 9   release_date          3000 non-null   object 
 10  runtime               3000 non-null   float64
 11  spoken_languages      3000 non-null   object 
 12  title                 3000 non-null   object 
 13  cast                  3000 non-null   object 
 14  crew                  3000 non-null   object 
 15  revenue              

In [47]:
df['budget_runtime'] = (df['budget'] + 1)/(df['runtime']+1)
df['poularity_year'] = (df['popularity']+1)/(df['year']+1)
df['budget_popularity'] = (df['budget']+1)/(df['popularity']+1)
df['budget_year'] = (df['budget'] + 1)/(df['year']+1)
df['runtime_year'] = (df['runtime']+1)/(df['year']+1)

#test


In [48]:
test['budget_runtime'] = (test['budget'] + 1)/(test['runtime']+1)
test['poularity_year'] = (test['popularity']+1)/(test['year']+1)
test['budget_popularity'] = (test['budget']+1)/(test['popularity']+1)
test['budget_year'] = (test['budget'] + 1)/(test['year']+1)
test['runtime_year'] = (test['runtime']+1)/(test['year']+1)

In [49]:
# want to add 
# number of crew/cast
# Production companies 
# possibly add day of week, day of year, month, or quarter 
# number of keywords 
# has homepage 
# original language? 
# 

In [50]:
df.head()

,id,budget,genres,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,...,revenue,year,budget_year_ratio,has_overview,overview_len,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,en,Hot Tub Time Machine 2,6.575393,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,...,16.326300,2015,3.448085,1,155,148936.180851,0.003758,1.848089e+06,6944.444940,0.046627
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,en,The Princess Diaries 2: Royal Engagement,8.248895,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,...,18.370959,2004,9.960120,1,393,350877.201754,0.004613,4.324841e+06,19950.125187,0.056858
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,en,Whiplash,64.299990,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,...,16.387512,2014,0.813570,1,130,31132.084906,0.032407,5.053601e+04,1637.717618,0.052605
3,4,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",tt1821480,hi,Kahaani,3.174936,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,...,16.588099,2012,0.296432,1,581,9756.105691,0.002074,2.874298e+05,596.125683,0.061103
4,5,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",tt1380152,ko,마린보이,1.148070,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,...,15.182615,2009,0.000000,1,168,0.008403,0.001069,4.655342e-01,0.000498,0.059204


In [51]:
#X = df[["popularity", "runtime", "budget", "budget_year_ratio", "year",'budget_runtime', 'poularity_year', 'budget_popularity','budget_year','runtime_year', 'has_overview', 'overview_len' ]]

In [52]:
X.head()

,popularity,runtime,budget,budget_year_ratio,year,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,has_overview,overview_len
0,6.575393,93.0,14000000,3.448085,2015,148936.180851,0.003758,1.848089e+06,6944.444940,0.046627,1,155
1,8.248895,113.0,40000000,9.960120,2004,350877.201754,0.004613,4.324841e+06,19950.125187,0.056858,1,393
2,64.299990,105.0,3300000,0.813570,2014,31132.084906,0.032407,5.053601e+04,1637.717618,0.052605,1,130
3,3.174936,122.0,1200000,0.296432,2012,9756.105691,0.002074,2.874298e+05,596.125683,0.061103,1,581
4,1.148070,118.0,0,0.000000,2009,0.008403,0.001069,4.655342e-01,0.000498,0.059204,1,168


In [53]:
#df['revenue'] = np.log1p(df['revenue'])
y = df[["revenue"]]

In [54]:
members = df["cast"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

df["num_cast"] = num_cast

In [55]:
members = df["crew"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

df["num_crew"] = num_cast

In [56]:
members = test["cast"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

test["num_cast"] = num_cast

In [57]:
members = test["crew"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

test["num_crew"] = num_cast

In [58]:
test.head()

,id,budget,genres,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,...,budget_year_ratio,has_overview,overview_len,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,num_cast,num_crew
0,3001,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",tt1226251,ja,ディアルガVSパルキアVSダークライ,3.851534,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",7/14/07,...,0.000000,1,335,0.010989,0.002416,0.206120,0.000498,0.045319,7,2
1,3002,88000,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",tt0051380,en,Attack of the 50 Foot Woman,3.559789,"[{'name': 'Woolner Brothers Pictures Inc.', 'i...","[{'iso_3166_1': 'US', 'name': 'United States o...",5/19/58,...,0.020777,1,162,1333.348485,0.002215,19299.357931,42.739679,0.032054,10,8
2,3003,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",tt0118556,en,Addicted to Love,8.085194,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",5/23/97,...,0.000000,1,362,0.009901,0.004547,0.110069,0.000501,0.050551,9,10
3,3004,6800000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",tt1255953,fr,Incendies,8.596012,"[{'name': 'TS Productions', 'id': 313}, {'name...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",9/4/10,...,1.683127,1,302,51908.404580,0.004772,708627.813304,3381.402785,0.065142,23,69
4,3005,2000000,"[{'id': 36, 'name': 'History'}, {'id': 99, 'na...",tt0418753,en,Inside Deep Throat,3.217680,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2/11/05,...,0.497509,1,894,21505.387097,0.002103,474194.580907,997.009472,0.046361,4,14


In [59]:
df.head()

,id,budget,genres,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,...,budget_year_ratio,has_overview,overview_len,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,num_cast,num_crew
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,en,Hot Tub Time Machine 2,6.575393,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,...,3.448085,1,155,148936.180851,0.003758,1.848089e+06,6944.444940,0.046627,24,72
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,en,The Princess Diaries 2: Royal Engagement,8.248895,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,...,9.960120,1,393,350877.201754,0.004613,4.324841e+06,19950.125187,0.056858,20,9
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,en,Whiplash,64.299990,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,...,0.813570,1,130,31132.084906,0.032407,5.053601e+04,1637.717618,0.052605,51,64
3,4,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",tt1821480,hi,Kahaani,3.174936,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,...,0.296432,1,581,9756.105691,0.002074,2.874298e+05,596.125683,0.061103,7,3
4,5,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",tt1380152,ko,마린보이,1.148070,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,...,0.000000,1,168,0.008403,0.001069,4.655342e-01,0.000498,0.059204,4,2


In [60]:
# Genre 

genres = df["genres"]

first = []
second = []
third = []
fourth = []
fifth = []
sixth = []
seventh = []

for string in genres:
    if isinstance(string, str):
        splits = string.split("'")
        genre1 = splits[5]
        first.append(genre1)
        multiples1 = splits[7:]
        if not multiples1:
            second.append('NA')
        else:
            genre2 = multiples1[4]
            second.append(genre2)
        multiples2 = multiples1[7:]
        if not multiples2:
            third.append('NA')
        else:
            genre3 = multiples2[3]
            third.append(genre3)
        multiples3 = multiples2[6:]
        if not multiples3:
            fourth.append('NA')
        else:
            genre4 = multiples3[3]
            fourth.append(genre4)
        multiples4 = multiples3[5:]
        if not multiples4:
            fifth.append('NA')
        else:
            genre5 = multiples4[4]
            fifth.append(genre5)
        multiples5 = multiples4[6:]
        if not multiples5:
            sixth.append('NA')
        else:
            genre6 = multiples5[4]
            sixth.append(genre6)
        multiples6 = multiples5[6:]
        if not multiples6:
            seventh.append('NA')
        else:
            genre7 = multiples6[4]
            seventh.append(genre7)
    else:
        first.append('NA')
        second.append('NA')
        third.append('NA')
        fourth.append('NA')
        fifth.append('NA')
        sixth.append('NA')
        seventh.append('NA')

genres_list = list(zip(first, second, third, fourth, fifth, sixth, seventh))
df["genres_list"] = genres_list


In [61]:

genresInMovie = df["genres_list"]

action = []
adventure = []
animation = []
comedy = []
crime = []
documentary = []
drama = []
family = []
fantasy = []
foreign = []
horror = []
music = []
mystery = []
romance = []
scifi = []
thriller = []

for movie in genresInMovie:
    movie = list(movie)
    
    isAction = 0
    isAdventure = 0
    isAnimation = 0
    isComedy = 0
    isCrime = 0
    isDocumentary = 0
    isDrama = 0
    isFamily = 0
    isFantasy = 0
    isForeign = 0
    isHorror = 0
    isMusic = 0
    isMystery = 0
    isRomance = 0
    isSciFi = 0
    isThriller = 0
    
    for genre in movie:
        if genre == "Action":
            isAction = 1
        if genre == "Adventure":
            isAdventure = 1
        if genre == "Animation":
            isAnimation = 1
        if genre == "Comedy":
            isComedy = 1
        if genre == "Crime":
            isCrime = 1
        if genre == "Documentary":
            isDocumentary = 1
        if genre == "Drama":
            isDrama = 1
        if genre == "Family":
            isFamily = 1
        if genre == "Fantasy":
            isFantasy = 1
        if genre == "Foreign":
            isForeign = 1
        if genre == "Horror":
            isHorror = 1
        if genre == "Music":
            isMusic = 1
        if genre == "Mystery":
            isMystery = 1
        if genre == "Romance":
            isRomance = 1
        if genre == "Science Fiction":
            isSciFi = 1
        if genre == "Thriller":
            isThriller = 1
            
    action.append(isAction)
    adventure.append(isAdventure)
    animation.append(isAnimation)
    comedy.append(isComedy)
    crime.append(isCrime)
    documentary.append(isDocumentary)
    drama.append(isDrama)
    family.append(isFamily)
    fantasy.append(isFantasy)
    foreign.append(isForeign)
    horror.append(isHorror)
    music.append(isMusic)
    mystery.append(isMystery)
    romance.append(isRomance)
    scifi.append(isSciFi)
    thriller.append(isThriller)
    
df["isAction"] = action
df["isAdventure"] = adventure
df["isAnimation"] = animation
df["isComedy"] = comedy
df["isCrime"] = crime
df["isDocumentary"] = documentary
df["isDrama"] = drama
df["isFamily"] = family
df["isFantasy"] = fantasy
df["isForeign"] = foreign
df["isHorror"] = horror
df["isMusic"] = music
df["isMystery"] = mystery
df["isRomance"] = romance
df["isSciFi"] = scifi
df["isThriller"] = thriller


In [98]:
genres = test["genres"]

first = []
second = []
third = []
fourth = []
fifth = []
sixth = []
seventh = []

for string in genres:
    if isinstance(string, str):
        splits = string.split("'")
        genre1 = splits[5]
        first.append(genre1)
        multiples1 = splits[7:]
        if not multiples1:
            second.append('NA')
        else:
            genre2 = multiples1[4]
            second.append(genre2)
        multiples2 = multiples1[7:]
        if not multiples2:
            third.append('NA')
        else:
            genre3 = multiples2[3]
            third.append(genre3)
        multiples3 = multiples2[6:]
        if not multiples3:
            fourth.append('NA')
        else:
            genre4 = multiples3[3]
            fourth.append(genre4)
        multiples4 = multiples3[5:]
        if not multiples4:
            fifth.append('NA')
        else:
            genre5 = multiples4[4]
            fifth.append(genre5)
        multiples5 = multiples4[6:]
        if not multiples5:
            sixth.append('NA')
        else:
            genre6 = multiples5[4]
            sixth.append(genre6)
        multiples6 = multiples5[6:]
        if not multiples6:
            seventh.append('NA')
        else:
            genre7 = multiples6[4]
            seventh.append(genre7)
    else:
        first.append('NA')
        second.append('NA')
        third.append('NA')
        fourth.append('NA')
        fifth.append('NA')
        sixth.append('NA')
        seventh.append('NA')

genres_list = list(zip(first, second, third, fourth, fifth, sixth, seventh))
test["genres_list"] = genres_list


In [99]:
genresInMovie = test["genres_list"]

action = []
adventure = []
animation = []
comedy = []
crime = []
documentary = []
drama = []
family = []
fantasy = []
foreign = []
horror = []
music = []
mystery = []
romance = []
scifi = []
thriller = []

for movie in genresInMovie:
    movie = list(movie)
    
    isAction = 0
    isAdventure = 0
    isAnimation = 0
    isComedy = 0
    isCrime = 0
    isDocumentary = 0
    isDrama = 0
    isFamily = 0
    isFantasy = 0
    isForeign = 0
    isHorror = 0
    isMusic = 0
    isMystery = 0
    isRomance = 0
    isSciFi = 0
    isThriller = 0
    
    for genre in movie:
        if genre == "Action":
            isAction = 1
        if genre == "Adventure":
            isAdventure = 1
        if genre == "Animation":
            isAnimation = 1
        if genre == "Comedy":
            isComedy = 1
        if genre == "Crime":
            isCrime = 1
        if genre == "Documentary":
            isDocumentary = 1
        if genre == "Drama":
            isDrama = 1
        if genre == "Family":
            isFamily = 1
        if genre == "Fantasy":
            isFantasy = 1
        if genre == "Foreign":
            isForeign = 1
        if genre == "Horror":
            isHorror = 1
        if genre == "Music":
            isMusic = 1
        if genre == "Mystery":
            isMystery = 1
        if genre == "Romance":
            isRomance = 1
        if genre == "Science Fiction":
            isSciFi = 1
        if genre == "Thriller":
            isThriller = 1
            
    action.append(isAction)
    adventure.append(isAdventure)
    animation.append(isAnimation)
    comedy.append(isComedy)
    crime.append(isCrime)
    documentary.append(isDocumentary)
    drama.append(isDrama)
    family.append(isFamily)
    fantasy.append(isFantasy)
    foreign.append(isForeign)
    horror.append(isHorror)
    music.append(isMusic)
    mystery.append(isMystery)
    romance.append(isRomance)
    scifi.append(isSciFi)
    thriller.append(isThriller)
    
test["isAction"] = action
test["isAdventure"] = adventure
test["isAnimation"] = animation
test["isComedy"] = comedy
test["isCrime"] = crime
test["isDocumentary"] = documentary
test["isDrama"] = drama
test["isFamily"] = family
test["isFantasy"] = fantasy
test["isForeign"] = foreign
test["isHorror"] = horror
test["isMusic"] = music
test["isMystery"] = mystery
test["isRomance"] = romance
test["isSciFi"] = scifi
test["isThriller"] = thriller

In [91]:
X = df[["popularity", "runtime", "budget", "budget_year_ratio", "year",'budget_runtime', 'poularity_year', 'budget_popularity','budget_year','runtime_year', 'has_overview', 'overview_len', "num_cast", 'num_crew',"isAction", "isComedy", "isCrime", "isDocumentary", 'isDrama', "isFamily","isFantasy", "isForeign", "isHorror", "isMusic", "isMystery","isRomance", "isSciFi", "isThriller" 
        ]]
# "isAction", "isComedy", "isCrime", "isDocumentary", 'isDrama', "isFamily","isFantasy", "isForeign", "isHorror", "isMusic", "isMystery","isRomance", "isSciFi", "isThriller"

In [63]:
X.head()

,popularity,runtime,budget,budget_year_ratio,year,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,...,isDrama,isFamily,isFantasy,isForeign,isHorror,isMusic,isMystery,isRomance,isSciFi,isThriller
0,6.575393,93.0,14000000,3.448085,2015,148936.180851,0.003758,1.848089e+06,6944.444940,0.046627,...,0,0,0,0,0,0,0,0,0,0
1,8.248895,113.0,40000000,9.960120,2004,350877.201754,0.004613,4.324841e+06,19950.125187,0.056858,...,1,1,0,0,0,0,0,1,0,0
2,64.299990,105.0,3300000,0.813570,2014,31132.084906,0.032407,5.053601e+04,1637.717618,0.052605,...,1,0,0,0,0,0,0,0,0,0
3,3.174936,122.0,1200000,0.296432,2012,9756.105691,0.002074,2.874298e+05,596.125683,0.061103,...,1,0,0,0,0,0,0,0,0,1
4,1.148070,118.0,0,0.000000,2009,0.008403,0.001069,4.655342e-01,0.000498,0.059204,...,0,0,0,0,0,0,0,0,0,1


In [64]:
#X, y, nas = proc_df(df, "revenue")
#df_trn, y_trn, nas = proc_df(df_raw, 'revenue')

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [93]:
X_train.shape, X_test.shape, y_test.shape, y_train.shape

((2400, 28), (600, 28), (600, 1), (2400, 1))

In [66]:
print(y)

        revenue
0     16.326300
1     18.370959
2     16.387512
3     16.588099
4     15.182615
...         ...
2995  14.283442
2996  12.103990
2997  18.309266
2998  18.962792
2999  18.223292

[3000 rows x 1 columns]


In [68]:
'''
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)
'''

'\nxgbr = xgb.XGBRegressor(verbosity=0) \nprint(xgbr)\n'

In [69]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2400 entries, 642 to 860
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   popularity         2400 non-null   float64
 1   runtime            2400 non-null   float64
 2   budget             2400 non-null   int64  
 3   budget_year_ratio  2400 non-null   float64
 4   year               2400 non-null   int64  
 5   budget_runtime     2400 non-null   float64
 6   poularity_year     2400 non-null   float64
 7   budget_popularity  2400 non-null   float64
 8   budget_year        2400 non-null   float64
 9   runtime_year       2400 non-null   float64
 10  has_overview       2400 non-null   int32  
 11  overview_len       2400 non-null   int64  
 12  num_cast           2400 non-null   int64  
 13  num_crew           2400 non-null   int64  
dtypes: float64(8), int32(1), int64(5)
memory usage: 271.9 KB


In [70]:
#xgbr.fit(X_train, y_train)

In [71]:
#ypred = xgbr.predict(X_test)
#mse = mean_squared_error(y_test, ypred)
#print("MSE: %.2f" % mse)
#MSE: 3.35
#print("RMSE: %.2f" % (mse**(1/2.0)))
#RMSE: 1.83 

In [75]:
m = RandomForestRegressor(n_estimators=124, random_state = 42, min_samples_leaf = 10,
                          n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)

C:\Users\Ben\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(min_samples_leaf=10, n_estimators=124, n_jobs=-1,
                      oob_score=True, random_state=42)

In [73]:
def rmse(y_pred, y_true):
    return np.sqrt(mean_squared_error(y_pred, y_true))

def print_rf_score(model):
    print(f'Train R2:   {model.score(X_train, y_train)}')
    print(f'Test R2:   {model.score(X_test, y_test)}')
    print(f'Train RMSE: {rmse(model.predict(X_train), y_train)}')
    print(f'Test RMSE: {rmse(model.predict(X_test), y_test)}')

In [76]:
print_rf_score(m)

Train R2:   0.6623335488342981
Test R2:   0.5225854533907768
Train RMSE: 1.7886550096745246
Test RMSE: 2.0627443053392494


In [94]:
xgb = XGBRegressor(
        n_estimators = 20000, 
        learning_rate = 0.001,
        max_depth = 6, 
        subsample = 0.3, 
        colsample_bytree = 0.2)
xgb.fit(X_train, y_train)

[13:36:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.2, learning_rate=0.001, max_depth=6,
             n_estimators=20000, subsample=0.3)

In [95]:
print_rf_score(xgb)

Train R2:   0.876902854718165
Test R2:   0.5371386096874748
Train RMSE: 1.0799573815489032
Test RMSE: 2.0310613917005447


In [100]:
predictdf= test[["popularity", "runtime", "budget", "budget_year_ratio", "year",'budget_runtime', 'poularity_year', 'budget_popularity','budget_year','runtime_year', 'has_overview', 'overview_len', "num_cast", 'num_crew',"isAction", "isComedy", "isCrime", "isDocumentary", 'isDrama', "isFamily","isFantasy", "isForeign", "isHorror", "isMusic", "isMystery","isRomance", "isSciFi", "isThriller" 
        ]]

In [78]:
predictdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   popularity         4398 non-null   float64
 1   runtime            4398 non-null   float64
 2   budget             4398 non-null   int64  
 3   budget_year_ratio  4398 non-null   float64
 4   year               4398 non-null   int64  
 5   budget_runtime     4398 non-null   float64
 6   poularity_year     4398 non-null   float64
 7   budget_popularity  4398 non-null   float64
 8   budget_year        4398 non-null   float64
 9   runtime_year       4398 non-null   float64
 10  has_overview       4398 non-null   int32  
 11  overview_len       4398 non-null   int64  
 12  num_cast           4398 non-null   int64  
 13  num_crew           4398 non-null   int64  
dtypes: float64(8), int32(1), int64(5)
memory usage: 464.0 KB


In [79]:
predictdf.head()

,popularity,runtime,budget,budget_year_ratio,year,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,has_overview,overview_len,num_cast,num_crew
0,3.851534,90.0,0,0.000000,2007,0.010989,0.002416,0.206120,0.000498,0.045319,1,335,7,2
1,3.559789,65.0,88000,0.020777,2058,1333.348485,0.002215,19299.357931,42.739679,0.032054,1,162,10,8
2,8.085194,100.0,0,0.000000,1997,0.009901,0.004547,0.110069,0.000501,0.050551,1,362,9,10
3,8.596012,130.0,6800000,1.683127,2010,51908.404580,0.004772,708627.813304,3381.402785,0.065142,1,302,23,69
4,3.217680,92.0,2000000,0.497509,2005,21505.387097,0.002103,474194.580907,997.009472,0.046361,1,894,4,14


In [80]:
#predictdf = np.nan_to_num(predictdf)

In [81]:
predictdf

,popularity,runtime,budget,budget_year_ratio,year,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,has_overview,overview_len,num_cast,num_crew
0,3.851534,90.0,0,0.000000,2007,0.010989,0.002416,2.061204e-01,0.000498,0.045319,1,335,7,2
1,3.559789,65.0,88000,0.020777,2058,1333.348485,0.002215,1.929936e+04,42.739679,0.032054,1,162,10,8
2,8.085194,100.0,0,0.000000,1997,0.009901,0.004547,1.100692e-01,0.000501,0.050551,1,362,9,10
3,8.596012,130.0,6800000,1.683127,2010,51908.404580,0.004772,7.086278e+05,3381.402785,0.065142,1,302,23,69
4,3.217680,92.0,2000000,0.497509,2005,21505.387097,0.002103,4.741946e+05,997.009472,0.046361,1,894,4,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4393,9.970359,118.0,42000000,10.489508,2001,352941.184874,0.005480,3.828498e+06,20979.021479,0.059441,1,151,10,11
4394,6.046516,95.0,19000000,4.731057,2004,197916.677083,0.003514,2.696368e+06,9476.309726,0.047880,1,457,22,94
4395,9.596883,129.0,16000000,4.072984,1982,123076.930769,0.005344,1.509878e+06,8068.583459,0.065557,1,504,17,12
4396,20.359336,100.0,2000000,0.492584,2015,19801.990099,0.010595,9.363592e+04,992.063988,0.050099,1,352,29,29


In [82]:
X_train.head()

,popularity,runtime,budget,budget_year_ratio,year,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,has_overview,overview_len,num_cast,num_crew
642,7.830087,116.0,39000000,9.868061,1988,333333.341880,0.004439,4.416718e+06,19607.843640,0.058824,1,284,14,12
700,7.409179,94.0,25000000,6.338427,1986,263157.905263,0.004232,2.972942e+06,12581.782084,0.047811,1,392,17,15
226,7.668829,105.0,50000000,12.363626,2011,471698.122642,0.004309,5.767792e+06,24850.895129,0.052684,1,443,51,70
1697,5.860319,132.0,0,0.000000,1992,0.007519,0.003442,1.457658e-01,0.000502,0.066734,1,458,12,6
1010,17.310647,125.0,35000000,8.620212,2015,277777.785714,0.009083,1.911456e+06,17361.111607,0.062500,1,135,34,22


In [83]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   popularity         3000 non-null   float64
 1   runtime            3000 non-null   float64
 2   budget             3000 non-null   int64  
 3   budget_year_ratio  3000 non-null   float64
 4   year               3000 non-null   int64  
 5   budget_runtime     3000 non-null   float64
 6   poularity_year     3000 non-null   float64
 7   budget_popularity  3000 non-null   float64
 8   budget_year        3000 non-null   float64
 9   runtime_year       3000 non-null   float64
 10  has_overview       3000 non-null   int32  
 11  overview_len       3000 non-null   int64  
 12  num_cast           3000 non-null   int64  
 13  num_crew           3000 non-null   int64  
dtypes: float64(8), int32(1), int64(5)
memory usage: 316.5 KB


In [84]:
np.isnan(predictdf)

,popularity,runtime,budget,budget_year_ratio,year,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,has_overview,overview_len,num_cast,num_crew
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4393,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4394,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4395,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4396,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [85]:
np.where(np.isnan(predictdf))
np.nan_to_num(predictdf)

array([[3.8515340e+00, 9.0000000e+01, 0.0000000e+00, ..., 3.3500000e+02,
        7.0000000e+00, 2.0000000e+00],
       [3.5597890e+00, 6.5000000e+01, 8.8000000e+04, ..., 1.6200000e+02,
        1.0000000e+01, 8.0000000e+00],
       [8.0851940e+00, 1.0000000e+02, 0.0000000e+00, ..., 3.6200000e+02,
        9.0000000e+00, 1.0000000e+01],
       ...,
       [9.5968830e+00, 1.2900000e+02, 1.6000000e+07, ..., 5.0400000e+02,
        1.7000000e+01, 1.2000000e+01],
       [2.0359336e+01, 1.0000000e+02, 2.0000000e+06, ..., 3.5200000e+02,
        2.9000000e+01, 2.9000000e+01],
       [1.1305910e+01, 8.5000000e+01, 6.4000000e+04, ..., 9.3000000e+01,
        1.9000000e+01, 2.5000000e+01]])

In [86]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    4398 non-null   int64  
 1   budget                4398 non-null   int64  
 2   genres                4398 non-null   object 
 3   imdb_id               4398 non-null   object 
 4   original_language     4398 non-null   object 
 5   original_title        4398 non-null   object 
 6   popularity            4398 non-null   float64
 7   production_companies  4398 non-null   object 
 8   production_countries  4398 non-null   object 
 9   release_date          4398 non-null   object 
 10  runtime               4398 non-null   float64
 11  spoken_languages      4398 non-null   object 
 12  title                 4398 non-null   object 
 13  cast                  4398 non-null   object 
 14  crew                  4398 non-null   object 
 15  year                 

In [87]:
predictdf

,popularity,runtime,budget,budget_year_ratio,year,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,has_overview,overview_len,num_cast,num_crew
0,3.851534,90.0,0,0.000000,2007,0.010989,0.002416,2.061204e-01,0.000498,0.045319,1,335,7,2
1,3.559789,65.0,88000,0.020777,2058,1333.348485,0.002215,1.929936e+04,42.739679,0.032054,1,162,10,8
2,8.085194,100.0,0,0.000000,1997,0.009901,0.004547,1.100692e-01,0.000501,0.050551,1,362,9,10
3,8.596012,130.0,6800000,1.683127,2010,51908.404580,0.004772,7.086278e+05,3381.402785,0.065142,1,302,23,69
4,3.217680,92.0,2000000,0.497509,2005,21505.387097,0.002103,4.741946e+05,997.009472,0.046361,1,894,4,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4393,9.970359,118.0,42000000,10.489508,2001,352941.184874,0.005480,3.828498e+06,20979.021479,0.059441,1,151,10,11
4394,6.046516,95.0,19000000,4.731057,2004,197916.677083,0.003514,2.696368e+06,9476.309726,0.047880,1,457,22,94
4395,9.596883,129.0,16000000,4.072984,1982,123076.930769,0.005344,1.509878e+06,8068.583459,0.065557,1,504,17,12
4396,20.359336,100.0,2000000,0.492584,2015,19801.990099,0.010595,9.363592e+04,992.063988,0.050099,1,352,29,29


In [88]:

submission = m.predict(predictdf)
submission


array([13.66009829, 13.33552566, 15.89599956, ..., 17.39348828,
       15.9809265 , 15.06654126])

In [101]:
xgbsubmission = xgb.predict(predictdf)
xgbsubmission

array([14.187193, 15.39902 , 16.109634, ..., 17.30421 , 15.969515,
       14.41738 ], dtype=float32)

In [102]:
subdf = pd.DataFrame(xgbsubmission)
subdf

,0
0,14.187193
1,15.399020
2,16.109634
3,16.574478
4,14.177845
...,...
4393,17.778255
4394,17.430113
4395,17.304211
4396,15.969515


In [103]:
subdf["id"] = test["id"]
subdf.columns = ['revenue', 'id']

In [104]:
subdf = subdf[['id', 'revenue']]

In [105]:
subdf['revenue'] = np.round(np.expm1(subdf["revenue"]))

In [106]:
subdf

,id,revenue
0,3001,1450171.0
1,3002,4872024.0
2,3003,9915743.0
3,3004,15783538.0
4,3005,1436678.0
...,...,...
4393,7394,52601516.0
4394,7395,37136572.0
4395,7396,32743356.0
4396,7397,8619303.0


In [107]:
subdf.to_csv("Submission3.csv")

In [ ]:
# month released 
# week released etc. 
# number of keywords 
# cast gender? 